In [ ]:
import os

In [ ]:
os.environ['OPENAI_API_KEY']='---'


In [ ]:
from langchain.llms import OpenAI


In [ ]:
!activeloop login -t '----'

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

# Before executing the following code, make sure to have your
# Activeloop key saved in the “ACTIVELOOP_TOKEN” environment variable.

# instantiate the LLM and embeddings models
embeddings = OpenAIEmbeddings()

# create our documents
from langchain.document_loaders import TextLoader

## here you can add your pdf file, cloned github repo, ...
root_dir='./somefile-or-gitrepo'
docs=[]

for dirpath, dirname, filename in os.walk(root_dir):
    for file in filename:
        try:
            loader=TextLoader(os.path.join(dirpath,file),encoding='utf-8')
            docs.extend(loader.load_and_split())
        except Exception as e:
            pass

# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
my_activeloop_org_id = "org-id"
my_activeloop_dataset_name = "textr3"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

from langchain.text_splitter import CharacterTextSplitter
text_splitter=CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts=text_splitter.split_documents(docs)

# add documents to our Deep Lake dataset
db.add_documents(texts)

In [ ]:
retriver=db.as_retriever()
retriver.search_kwargs['distance_metric']='cos'
retriver.search_kwargs['fetch_k']=10000
retriver.search_kwargs['maximal_marginal_relavance']=True
retriver.search_kwargs['k']=10

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

model= ChatOpenAI(model='gpt-4') # 'gpt-3.5-turbo',

qa=ConversationalRetrievalChain.from_llm(model,retriver)

In [ ]:
import textwrap

questions=[
    "where these entity and edgrs are indexed and then calculate the score"
]
chat_history=[]
for question in questions:
    result=qa({"question":question,"chat_history":chat_history})
    chat_history.append({question,result['answer']})
    print(f"question ----> {question} \n")
    wrapped_text = textwrap.wrap(str(result['chat_history'][0]), width=100)
    for line in wrapped_text:
        print(f"{line}")

In [ ]:
import textwrap

questions=[
 "what dose Graph.java"
]
chat_history=[]
for question in questions:
    result=qa({"question":question,"chat_history":chat_history})
    chat_history.append({question,result['answer']})
    print(f"question ----> {question} \n")
    wrapped_text = textwrap.wrap(str(result['chat_history'][0]), width=100)
    for line in wrapped_text:
        print(f"{line}")

In [ ]:
# load the existing Deep Lake dataset and specify the embedding function
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# create new documents
texts = [
    "Lady Gaga was born in 28 March 1986",
    "Michael Jeffrey Jordan was born in 17 February 1963"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# add documents to our Deep Lake dataset
db.add_documents(docs)

In [ ]:
# instantiate the wrapper class for GPT3
llm = OpenAI(model="text-davinci-003", temperature=0)

# create a retriever from the db
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm, chain_type="stuff", retriever=db.as_retriever()
)

# instantiate a tool that uses the retriever
tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

# create an agent that uses the tool
agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)

In [ ]:
response = agent.run("When was Michael Jordan born?")
print(response)

In [ ]:
from langchain.llms import OpenAI

from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain.agents import initialize_agent

from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper

In [ ]:
llm = OpenAI(model="text-davinci-003", temperature=0)


In [ ]:
GOOGLE_API_KEY=""
api_key = os.environ["GOOGLE_API_KEY"]


In [ ]:
#search = GoogleSearchAPIWrapper()

In [ ]:
#tools = [
#    Tool(
#        name = "google-search",
#        func=search.run,
#        description="useful for when you need to search google to answer questions about current events"
#    )
#]

In [ ]:
agent = initialize_agent(tools, 
                         llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True,
                         max_iterations=6)

In [ ]:
#response = agent("What's the latest news about the Mars rover?")
#print(response['output'])

In [ ]:
from langchain.llms import OpenAI
from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import initialize_agent, AgentType

In [ ]:
llm = OpenAI(model="text-davinci-003", temperature=0)

prompt = PromptTemplate(
    input_variables=["query"],
    template="Write a summary of the following text: {query}"
)

summarize_chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
# remember to set the environment variables
# “GOOGLE_API_KEY” and “GOOGLE_CSE_ID” to be able to use
# Google Search via API.
search = GoogleSearchAPIWrapper()

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for finding information about recent events"
    ),
    Tool(
       name='Summarizer',
       func=summarize_chain.run,
       description='useful for summarizing texts'
    )
]

In [ ]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  
)

In [ ]:
response = agent("What's the latest news about the Mars rover? Then please summarize the results.")
print(response['output'])